In [1]:
from langchain_community.graphs import Neo4jGraph
import os
from langchain_ollama import ChatOllama
import json
from datetime import datetime
from dotenv import load_dotenv
import os
from openai import OpenAI

In [2]:
load_dotenv()
graph = Neo4jGraph()

C:\Users\jocke\AppData\Local\Temp\ipykernel_61840\278928253.py:2: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph()


# Collect results

In [19]:
def get_nodes_by_topic_title(topic_title: str):
    """
    Returns nodes from the Neo4j database where the 'topic_title' property matches the given value.
    """
    query = """
       MATCH (p:Post)-[]-(a:Argument)-[]-(mn:MaxNeefCategory)
       WHERE p.topic_title = $topic_title
       RETURN p.id AS post_id,
       a.description AS descriptions,
       a.motivations_descriptions AS motivations,
       collect(DISTINCT mn.id) AS categories
       """
    
    results = graph.query(query, params={"topic_title": topic_title})
    return results #[record['n'] for record in results]

In [15]:
results = []

In [24]:

results = get_nodes_by_topic_title("test")

In [25]:
print("Results:")
for record in results:
    print(record)

Results:
{'post_id': 'test', 'descriptions': 'The design and architecture are sleek and modern', 'motivations': ['Appreciation for modern design and architecture due to their aesthetic appeal.'], 'categories': ['Creativity']}
{'post_id': 'test', 'descriptions': "It will be a fantastic addition to our city's infrastructure.", 'motivations': ['Wants to improve the quality of life in the city.'], 'categories': ['Participation']}
{'post_id': 'test', 'descriptions': 'The environmental impact assessments have clearly had a significant input in minimizing its footprint, which is great news for us locals who care about preserving our natural habitats.', 'motivations': ['Value for environmental protection and the role of EIAs in minimizing ecological impact, which benefits local communities who care about preserving nature.'], 'categories': ['Protection', 'Participation']}
{'post_id': 'test', 'descriptions': "It's clear that they're committed to making this airport a success for both the city a

In [36]:
post = next((item for item in results if item['post_id'] == "msw2stv"), None)
print(post)

{'post_id': 'msw2stv', 'descriptions': 'The President should have a clear and self-regulating mandate', 'motivations': ['Wants to ensure executive function is effective', 'Desires a system that prevents gerrymandering'], 'categories': ['Protection', 'Participation']}


# Retrieve context

In [ ]:
summarizer = ChatOllama(model="llama3.1")

resumos_cache = {} # To store summaries of previous posts

# Estimate tokens based on word count
def estimar_tokens(texto: str) -> int:
    return int(len(texto.split()) * 1.3)  # reasonable approximation

# Function to fetch the parents of a post (up to 3 levels)
def get_parent_posts(post_id: str, max_levels: int = 3):
    query = f"""
    MATCH (child:Post {{id: $post_id}})-[:RESPONDS_TO*1..{max_levels}]->(parent)
    WHERE (parent:Post OR parent:OriginalPost) AND toLower(parent.text) <> '[removed]'
    RETURN parent.text AS text, parent.id AS id
    ORDER BY size(parent.text) DESC
    """
    results = graph.query(query, params={"post_id": post_id})
    return [{"id": row["id"], "text": row["text"]} for row in results]

# Function to fetch the text of the current post
def get_post_text(post_id: str):
    query = """
    MATCH (p:Post {id: $post_id})
    RETURN p.text AS text
    """
    result = graph.query(query, params={"post_id": post_id})
    if result:
        return result[0]["text"]
    return None

# Function to summarize text with LLM, checking size and using cache
def resumir_texto(texto: str, token_threshold: int = 150) -> str:
    if texto in resumos_cache:
        return resumos_cache[texto]
    
    if estimar_tokens(texto) <= token_threshold:
        resumo = texto.strip()
    else:
        prompt = f"""
        Summarize the following post in a concise and informative way. If you refer to the author, make sure you refer to them as 'an author of previous posts'. Only keep what is essential to understand the point made, and return only the summary:

        \"\"\"{texto}\"\"\"
        """
        resumo = summarizer.invoke(prompt).content.strip()
    
    resumos_cache[texto] = resumo
    return resumo

# Main function to build summarized context from previous posts
def get_context(post_id: str, max_pais: int = 3, token_threshold: int = 200):
    pais = get_parent_posts(post_id, max_levels=max_pais)
    post_filho = get_post_text(post_id)

    if not post_filho:
        raise ValueError("Child post not found.")

    contexto_resumido = []
    for p in pais[:max_pais]:
        resumo = resumir_texto(p["text"], token_threshold=token_threshold)
        contexto_resumido.append(f"*Context from previous user's comment:* {resumo}")

    contexto_final = "\n".join(contexto_resumido)
    return contexto_final

# Instantiate model

In [ ]:
# CONFIGURATIONS (you can adjust this as preferred)
USE_OPENAI = False  # True for OpenAI, False for Ollama
OPENAI_MODEL = "gpt-4o"
OLLAMA_MODEL = "deepseek-r1:8b"

# OpenAI Client (only if used)
openai_api_key = os.getenv("OPENAI_API_KEY") or "INSERT_YOUR_KEY_HERE"
openai_client = OpenAI(api_key=openai_api_key)

ollama_llm = ChatOllama(model=OLLAMA_MODEL, temperature=0, format="json")

prompt_template = """
You are an evaluator specialized in assessing the credibility of automatic argument extraction from discussion posts.

Below is a post and the extracted information by a language model:

Original Post:
"{post_text}"

Extracted:
- Argument: {argument}
- Motivations: {motivation}
- Max-Neef Categories: {categories}


Max-Neef Categories refer to fundamental human needs based on Manfred Max-Neef's Human Scale Development theory. Each motivation should reflect one or more of these needs:

1. **Subsistence** - physical health, food, shelter.
2. **Protection** - safety, care, social security.
3. **Affection** - relationships, love, friendship.
4. **Understanding** - curiosity, education, knowledge.
5. **Participation** - involvement, responsibility, belonging.
6. **Leisure** - rest, fun, play, recreation.
7. **Creativity** - innovation, self-expression, skills.
8. **Identity** - sense of self, belonging, cultural roots.
9. **Freedom** - autonomy, choice, equality.

The arguments are to be simple bullet points, and the motivations are to be simple phrases associated with the arguments. The Max-Neef Categories are a list of categories categorizing the motivations with Max-Neef's theory. **Not every category needs to be present in each case**.

Evaluate the following criteria:
1. Is the argument extraction coherent with the post, reflecting the author's opinion?
2. Do the motivations seem like a plausible justification for why the author made the argument?
3. According to Max-Neef's Human Scale Development theory, do the categories accurately reflect the necessities associated with the defined motivations and arguments? How many categories are correctly identified?

Assign a score for each criterion, and write brief feedback.

Respond in the following JSON format:
{{
  "argument_extraction": 0-5, (where 0 means the argument is not coherent with the post, and 5 means it is very coherent)
  "motivation_plausibility": 0-5, (where 0 means the motivations are not plausible, and 5 means they are very plausible)
  "category_score": 0 - 100, (where 0 means you consider none of the categories are correct, and 100 means you consider all categories correct)
  "feedback": "Brief comment on strengths or weaknesses of the extraction. Ideal Max-Neef categorization."
}}

"""

def generate_evaluation_response(post_text, argument, motivation, categories, use_openai=USE_OPENAI):

    prompt = prompt_template.format(
        post_text=post_text.strip(),
        argument=argument.strip(),
        motivation=motivation,
        categories=categories
    )    
        
    if use_openai:
        print(f"🔗 Using OpenAI ({OPENAI_MODEL})...")
        response = openai_client.chat.completions.create(
            model=OPENAI_MODEL,
            messages=[
                {"role": "system", "content": "You are a critical evaluator of argument and motivation extraction."},
                {"role": "user", "content": prompt}
            ],
            temperature=0,
        )
        return response.choices[0].message.content
    
    else:
        print(f"💻 Using local model via Ollama ({OLLAMA_MODEL})...")       
        response = ollama_llm.invoke(prompt)
            
        return response.content

# Evaluate

In [28]:
id = results[0].get("post_id")
id

'test'

In [8]:
id = 'test'

In [ ]:
# Collect context
contexto = get_context(id, max_pais=3, token_threshold=200)

comentario = get_post_text(id)

text_input = contexto + "\n\n*Comment to analyze:* " + comentario

print(text_input)



*Comment to analyze:* I just wanted to share my thoughts on the new airport development in our area. As someone who has been following
the progress closely, I'm thoroughly impressed with how the project is coming along.

The design and architecture are sleek and modern, and I think it will be a fantastic addition to our city's
infrastructure. The environmental impact assessments have clearly had a significant input in minimizing its
footprint, which is great news for us locals who care about preserving our natural habitats.

I'm also impressed with the transparency of the project managers - they've been doing a great job of keeping us
informed through regular updates and open forums. It's clear that they're committed to making this airport a
success for both the city and the local community.

One thing I'd love to see in the future is more details about the public transportation links and amenities that
will be available to passengers. As someone who regularly commutes, it would be f

In [34]:
results[0].get("categories")

['Creativity']

In [ ]:
resposta = generate_evaluation_response(
    post_text=text_input,
    argument=results[0].get("descriptions"),
    motivation=results[0].get("motivations"),
    categories=results[0].get("categories")
)

print("📤 Evaluator's response:\n", resposta)

🔗 A usar OpenAI (gpt-4o)...
📤 Resposta do avaliador:
 ```json
{
  "argument_extraction": 4,
  "motivation_plausability": 3,
  "category_score": 50,
  "feedback": "The argument extraction is mostly coherent with the post, as the author does express appreciation for the design and architecture. However, the argument could be more comprehensive by including the author's positive view on the environmental impact and transparency aspects. The motivation of 'appreciation for modern design and architecture due to their aesthetic appeal' is plausible but somewhat limited, as the author also values the project's environmental considerations and community engagement. The Max-Neef category of 'Creativity' is partially correct, as it reflects the appreciation for design, but 'Understanding' could also be relevant due to the author's interest in the project's transparency and environmental impact. Including 'Participation' might also be appropriate given the author's appreciation for community enga

In [ ]:
# Prepare filename with model name and current date
model_name = OLLAMA_MODEL if not USE_OPENAI else OPENAI_MODEL
date_str = datetime.now().strftime("%Y%m%d")
filename = f"resultados_{model_name}_{date_str}.json"

# Parse resposta string to dict if needed
if isinstance(resposta, str):
    try:
        resposta_dict = json.loads(resposta)
    except json.JSONDecodeError:
        resposta_dict = {"resposta": resposta}
else:
    resposta_dict = resposta

# Add model name to the exported data
export_data = {
    "model_name": model_name,
    "result": resposta_dict
}

# Ensure the "avaliacoes" folder exists
os.makedirs("avaliacoes", exist_ok=True)
output_path = os.path.join("avaliacoes", filename)

# Export to JSON file
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(export_data, f, ensure_ascii=False, indent=2)

print(f"Exported to {output_path}")

Exported to avaliacoes\resultados_gpt-4o_20250518.json
